# **Data 200 - Final Project**


Author:  Aryan Jain

## **Summary Of Contents**

### This Final Project conatins the following sections :

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
